<a href="https://colab.research.google.com/github/DibyaSadhukhan/Webscraping/blob/main/Imdb_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
df = pd.DataFrame(columns=('Poster','Title', 'Year','Certificate','Runtime','genre','rating','Votes','starring','stars_links','direction','director_links','gross'))


In [2]:
links=['https://www.imdb.com/search/title/?user_rating=1.0,10.0&sort=num_votes,desc']
for i in range(51,50000,50):
  links.append('https://www.imdb.com/search/title/?user_rating=1.0,10.0&sort=num_votes,desc&start='+str(i)+'&ref_=adv_nxt')
  #links.append('https://www.imdb.com/search/title/?genres=action&sort=num_votes,desc&start='+str(i)+'&explore=title_type,genres&ref_=adv_nxt')
#print(links)

In [3]:
def getData(link,df):
  headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
  webPage = requests.get(link, headers = headers).text
  soup = BeautifulSoup(webPage, 'lxml')
  for tags in soup.find_all('div',class_ = 'lister-item mode-advanced'):
    try:
      Poster=tags.find('img',class_='loadlate')['src']
    except:
      Poster=""
    Title=tags.find('h3',class_='lister-item-header').text.strip().split('\n')[1]
    try:
      year=tags.find('h3',class_='lister-item-header').text.strip().split('\n')[2]
      
    except:
      year=""
    try:
      certificate=tags.find('span',class_='certificate').text
    except:
      certificate="N/A"
    try:
      runtime=tags.find('span',class_='runtime').text
    except:
      runtime="N/A"
    
    genre=tags.find('span',class_='genre').text.strip()
    rating=tags.find('div',class_='inline-block ratings-imdb-rating').text.strip()
    director=stars=''
    directorL=starsL=''
    for a in tags.find_all('a'):
        link=str(a.get('href'))
        if(link.find('dr_')!=-1):
            directorL+="https://www.imdb.com/"+link+" , "
            director+=str(a.text)+' , '
        elif(link.find('st_')!=-1):
            starsL+="https://www.imdb.com/"+link+" , "
            stars+=str(a.text)+' , '
    votes=tags.find('p',class_='sort-num_votes-visible').getText()
    votes=votes.split('\n')
    d=''
    for casts in votes:
        if len(casts.strip())>2:
            d=d+casts.strip()+' '
        if d.find('Gross:')!=-1:
            gross=d[(d.find('Gross:')+6):]
        else:
            gross=''
        if d.find('Votes:')!=-1:
            if d.find('Gross:')!=-1:
                votes=d[(d.find('Votes:')+6):(d.find('Gross:'))].strip()
                votes=votes[:-1]
            else:
                votes=d[(d.find('Votes:')+6):].strip()
                votes=votes[:-1]
    index=int((tags.find('h3',class_='lister-item-header').text.strip().split('\n')[0][:-1]).replace(',',''))
    df.loc[index] = [Poster,Title,year,certificate,runtime,genre,rating,votes,stars,starsL,director,directorL,gross]
  return df
  

In [4]:
count=1
for link in links:
  print("writing index ",count)
  df=getData(link,df)
  count+=50

  #df=getData(links[0],df)


writing index  1
writing index  51
writing index  101
writing index  151
writing index  201
writing index  251
writing index  301
writing index  351
writing index  401
writing index  451
writing index  501
writing index  551
writing index  601
writing index  651
writing index  701
writing index  751
writing index  801
writing index  851
writing index  901
writing index  951
writing index  1001
writing index  1051
writing index  1101
writing index  1151
writing index  1201
writing index  1251
writing index  1301
writing index  1351
writing index  1401
writing index  1451
writing index  1501
writing index  1551
writing index  1601
writing index  1651
writing index  1701
writing index  1751
writing index  1801
writing index  1851
writing index  1901
writing index  1951
writing index  2001
writing index  2051
writing index  2101
writing index  2151
writing index  2201
writing index  2251
writing index  2301
writing index  2351
writing index  2401
writing index  2451
writing index  2501
wri

In [5]:
print(df.shape)

(9998, 13)


In [6]:
df.to_csv('testdata.csv',encoding='utf-8-sig')